# Old Data Cleaning Code

This file contains code initially written to clean the data - specifically, to identify and remove follow-up reports.

In [2]:
# # This approach is too much for initial stages - what we want to do is actually understand
# # what markers of a "relief distributed" report look like. 

# # We want to remove, or tag the "Relief distributed" titles.
# from sklearn.feature_extraction.text import CountVectorizer

# train_simple = ['Relief distributed',
#                'Relief distibuted',
#                'Relief distributed in']


In [3]:
# vect = CountVectorizer(decode_error = 'ignore')
# vect.fit(train_simple)
# vect.get_feature_names()

In [4]:
# transform training data into a 'document-term matrix'
# train_simple_dtm = vect.transform(train_simple)
# train_simple_dtm.toarray()

In [5]:
# examine the vocabulary and document-term matrix together
# pd.DataFrame(train_simple_dtm.toarray(), columns=vect.get_feature_names())

In [6]:
# transform testing data into a document-term matrix (using existing vocabulary)
# test_simple = ["relief distributed"]
# test_simple_dtm = vect.transform(test_simple)
# test_simple_dtm.toarray()
# pd.DataFrame(test_simple_dtm.toarray(), columns=vect.get_feature_names())

In [7]:
# instantiate the vectorizer ( use variable name as vect)
# vect = CountVectorizer(decode_error = 'ignore')
# vect.fit(data['INCIDENT TITLE'])
# vect.get_feature_names()

In [8]:
# train_dtm = vect.transform(data['INCIDENT TITLE'])
# print train_dtm

In [9]:
# train_features = vect.get_feature_names()
# len(train_features)

In [10]:
# Explore the features of our dataset to figure out where the "reliefs" show up.
# Range[100:178] - all numbers
# Range[709:718] - pulls out all variations of dispatch + distribution
# train_features[-541:-540] - relief
# train_features[:219]

In [11]:
# train_features[-200:]

In [12]:
# convert train_dtm to a regular array
# train_arr = train_dtm.toarray()
# train_arr

In [13]:
# Create a DataFrame of tokens with their counts.
# train_token_counts = pd.DataFrame({'token':train_features, 'count':np.sum(train_arr, axis=0)})
# train_token_counts.iloc[-541]

In [14]:
# train_token_counts.iloc[709:718]

# How do I know whether a report is follow-up indicating that relief has been distributed?

1. Category column: this will include different categories a report's been tagged as. We'd want ones with "Distribution Area." Usually that occurs as the first or second category.

2. Incident title: "relief distributed" will feature in the incident title, to flag that this is a follow-up report about action taken. 

In [15]:
# Sample 50 rows at random to see what the prevalence of relief distribution reports are, potential identifiers of such reports
pd.options.display.max_colwidth = 500
sample = data.sample(n=50, replace=False)
sample

NameError: name 'pd' is not defined

In [16]:
# Calculate % of random sample that had relief distribution reports

float(5) / 50

0.1

In [17]:
# What does the category typically include for follow-up reports?
data.category.head()



NameError: name 'data' is not defined

In [ ]:
# Use the dot-split method to create a list out of the category text (instead of having it just be a string)
data['distarea_tag'] = data.CATEGORY.apply(lambda x:x.split(", "))

# REVISIT: add the column in next to the original CATEGORY column, so it's easy to visually check that this code has worked.

In [ ]:
# Use the "in" function to determine if the list of category tags for each row includes "Distribution Area"
# If it does, then a "True" is returned into the distarea_tag_dummy column 
# data['distarea_tag_dummy'] = data.distarea_tag.apply(lambda x: 'Distribution Area' in x)
# data.head()

In [ ]:
# Determine how many reports we tagged (that included Distribution Area)
# data.distarea_tag_dummy.value_counts()

In [ ]:
# Do a quick check of the tagged cases - first 50.
# pd.options.display.max_colwidth = 500
# data[['caseID', 'INCIDENT TITLE', 'DESCRIPTION', 'distarea_tag', 'distarea_tag_dummy']][data.distarea_tag_dummy == True].iloc[:50]

# A few false positives:
# 1950, 617, 303 (had relief needs + distribution area), 299, 182. 

In [ ]:
# Do a quick check of the tagged cases - first 50.
# pd.options.display.max_colwidth = 500
# data[['caseID', 'INCIDENT TITLE', 'DESCRIPTION', 'distarea_tag', 'distarea_tag_dummy']][data.distarea_tag_dummy == True].iloc[50:]

# 1271, 1259, 946, 886, 1049, 734, 617, 303, 299, 

In [ ]:
# False positives: 14. 13% (out of 108, out of 2035)

# data[['#', 'INCIDENT TITLE', 'DESCRIPTION']][data['INCIDENT TITLE'] == 'Help needed at Kalthari in Sindhupalchok']

In [ ]:
# Use the dot-split method to create a list out of the incident title text (instead of having it just be a string)
# data['incident_tag'] = data['INCIDENT TITLE'].apply(lambda x:x.split(", "))
# data

# look up documentation on str.contains
data['incident_tag'] = data['INCIDENT TITLE'].str.contains('Relief distributed')
data
data.incident_tag.value_counts()

In [ ]:
# If it does, then a "True" is returned into the distarea_tag_dummy column 
# data['incident_tag_d'] = data.incident_tag.apply(lambda x: 'Relief distributed' in x)
# data.head()

In [ ]:
data.incident_tag_d.value_counts()

In [ ]:
# Only keep the cases where Distribution Area was not included in the category tags. 
# The goal is to eliminate cases that were relief distribution reports, not new reports. 
data2 = data[data.distarea_tag_dummy==False]
data2

In [ ]:
# Check 2: How many remaining cases of "relief distributed" reports do we have? 
# Take another random sample 
data2.sample(n=50, replace=False)

In [ ]:
# Cases not caught:
# 2265 - CWIN Earthquake, 1813, 1911, 
# Trip reports: most likely these just contain status reports that could be a mix. 

In [ ]:
# How many cases not caught this time?
float(3)/50

# Down 40%.